In [1]:
import numpy as np
from gurobipy import * 

In [2]:
#useful info
# index sets: i plant, j warehouse, k retailer, y year
# 
demand = [1000,1200,1800,1200,1000,1400,1600,1000] 
rate = [0.2,0.2,0.25,0.2,0.2,0.25,0.25,0.2]
# t in range(2,11):
#    demand_1 = []
#    for j in range(len(demand[0])):
#        demand_1.append(demand[0][j]+demand[0][j]*rate[j]**(t-1))
#    demand.append(demand_1)

# Capacity
Capacity = [16000,12000,14000,10000,13000]
plant = [1,2,3,4,5]
warehouse = [1,2,3,4]
retail = [1,2,3,4,5,6,7,8]
MaxI = 4000
# construction costs 
cstr = [[2000000,1600000,1800000,900000,1500000]]

for i in range(2,11):
    c = []
    for j in range(len(cstr[0])):
        c.append(cstr[0][j]*(1.03)**(i-1))
    cstr.append(c)

# reopen costs (plant i in year l
reopen = [[190000,150000,160000,100000,130000]]
for i in range(2,11):
    c = []
    for j in range(len(reopen[0])):
        c.append(reopen[0][j]*(1.03)**(i-1))
    reopen.append(c)
# operating costs (plant i in year l
operate = [[420000,380000,460000,280000,340000]]
for i in range(2,11):
    c = []
    for j in range(len(operate[0])):
        c.append(operate[0][j]*(1.03)**(i-1))
    operate.append(c)

# shut down costs (plant i in year l
shut_down = [[170000,120000,130000,80000,110000]]
for i in range(2,11):
    c = []
    for j in range(len(shut_down[0])):
        c.append(shut_down[0][j]*(1.03)**(i-1))
    shut_down.append(c)

# shipping costs
## plant i to Warehouse j, i=5,j=4
P_WH_2=[];P_WH_3=[];P_WH_4=[];P_WH_5=[];P_WH_6=[];P_WH_7=[];P_WH_8=[];P_WH_9=[]; P_WH_10=[]
P_WH_1 = [[120,130,80,50],
          [100,30,100,90],
          [50,70,60,30],
          [60,30,70,70],
          [60,20,40,80]]
P_to_WH = [P_WH_1,P_WH_2,P_WH_3,P_WH_4,P_WH_5,P_WH_6,P_WH_7,P_WH_8,P_WH_9,P_WH_10]
for a in range(len(P_to_WH)-1):
    for i in range(len(plant)):
        k = []
        for j in range(len(warehouse)):
            k.append(P_WH_1[i][j]*(1.03)**(a+1))
        P_to_WH[a+1].append(k)

## warehouse i to retail k
WH_R_1 = [[90,100,60,50,80,90,20,120],
           [50,70,120,40,30,90,30,80],
           [60,90,70,90,90,40,110,70],
           [70,80,90,60,100,70,60,90]]

WH_R_2=[];WH_R_3=[];WH_R_4=[];WH_R_5=[];WH_R_6=[];WH_R_7=[];WH_R_8=[];WH_R_9=[];WH_R_10=[]
WH_to_R = [WH_R_1,WH_R_2,WH_R_3,WH_R_4,WH_R_5,WH_R_6,WH_R_7,WH_R_8,WH_R_9,WH_R_10]

for a in range(len(WH_to_R)-1):
    for i in range(len(warehouse)):
        k = []
        for j in range(len(retail)):
            k.append(WH_R_1[i][j]*(1.03)**(a+1))
        WH_to_R[a+1].append(k)

year = list(range(1,11))
print(year)
AC = [200] #alloy cost for each year through 10 years
for t in year:
    AC.append(AC[0]+(1.03)**t)
maxalloy = 60000


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [3]:
## setting variables
# construction binary, reopen binary, operating binary, shut down binary
# number of products produces in plant i year 
model1 = Model("XYZ company year 1")

# construction costs* construction binary c + reopen costs * reopen binary r + oeprating costs * operating binary o + shut down costs * shut down binary s
# + planttoWH shipping*plant to WH binary + WHtoretail shipping* WH retail binary
# + material costs 

## binary variables
C = model1.addVars(range(1,len(plant)+1),vtype=GRB.BINARY,ub=1,obj = cstr[0], name="construction") # construction 
O = model1.addVars(range(1,len(plant)+1),vtype=GRB.BINARY,ub=1,obj = operate[0], name="Operating") # operation
R = model1.addVars(range(1,len(plant)+1),vtype=GRB.BINARY,ub=1,obj = reopen[0], name="Reopening") # Reopen
S = model1.addVars(range(1,len(plant)+1),vtype=GRB.BINARY,ub=1,obj = shut_down[0], name="Shut down") # Shut down 
# lambda
#for i in range(len(year)): (warehouse variables)
X = model1.addVars(range(1,len(plant)+1), range(1,len(warehouse)+1),vtype=GRB.CONTINUOUS,obj = P_to_WH[0], name="ship_plant to WH") # plant to WH shipping 
Y = model1.addVars(range(1,len(warehouse)+1), range(1,len(retail)+1),vtype=GRB.CONTINUOUS,obj = WH_to_R[0], name="ship_WH to retail") # WH to retail shipping
I = model1.addVars(warehouse,vtype=GRB.CONTINUOUS, name = "warehouse Inventory keeping") # warehouse k in year i each year
leftover = model1.addVars(warehouse, vtype=GRB.CONTINUOUS, name ='remaining products in the warehouse')

#production variable set:
#Plantprod = model.addVars(plant,year,vtype=GRB.CONTINUOUS, name = "plantproduce")  # each year 

# Material variables:
mu1 = model1.addVars(plant,vtype=GRB.BINARY,name ="mu1") #widget
mu2 = model1.addVars(plant,vtype=GRB.BINARY,name ="mu2")  
mu3 = model1.addVars(plant,vtype=GRB.BINARY,name ="mu3")
z1 = model1.addVars(plant,vtype=GRB.BINARY,name ="z1")
z2 = model1.addVars(plant,vtype=GRB.BINARY,name ="z2")

model1.modelSense = GRB.MINIMIZE


Academic license - for non-commercial use only - expires 2022-09-19
Using license file C:\Users\mengt\gurobi.lic


## constraints

In [9]:

## binary constraints

#model.addConstrs(C[i][j] >= O[i][j], "Construction") # only constructed before operating 
model1.addConstrs((R[i] == O[i] for i in plant), "operation year 1")

## plants i to warehouse j in year y shipping & capacity constraints
for i in plant:
    model1.addConstr((sum(X[i,j] for j in warehouse) == Capacity[i-1]*O[i]), "plant shipping constriants")


### widget mu (plant, year)
for i in plant:
    model1.addConstr(sum(X[i,j] for j in warehouse)*3==(0*mu1[i]+(9000*150)*mu2[i]+(16000*3*120)*mu3[i]),"widget for plant i")
    model1.addConstr((mu1[i]+mu2[i]+mu3[i]==1), "control wiget")
    model1.addConstr((z1[i]+z2[i]==1), "control wiget assembly")
    model1.addConstr((mu1[i]<=z1[i]), "control wiget assembly")
    model1.addConstr((mu2[i]<=z1[i]+z2[i]), "control wiget assembly")
    model1.addConstr((mu3[i]<=z2[i]), "control wiget assembly")


model1.optimize()
print(model1.ObjVal)
for var in model1.getVars():
    if(var.x>0):
        print(var.varName,var.x)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 173 rows, 105 columns and 792 nonzeros
Model fingerprint: 0x0ac5945e
Variable types: 60 continuous, 45 integer (45 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+06]
  Objective range  [2e+01, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+04]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint control_wiget by 1.000000000

Presolve removed 131 rows and 64 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


AttributeError: Unable to retrieve attribute 'ObjVal'

In [4]:
model1.addConstrs((Y.sum('*',k) == demand[k-1] for k in retail), "Demand year 1") #demand constraint
#model1.addConstrs((X.sum('*', j) == Y.sum(j,'*') for j in warehouse),"flow conservation")
model1.addConstrs((X.sum('*', j) - leftover[j] == Y.sum(j,'*') for j in warehouse),"flow conservation year 1")

# inventory constraint
model1.addConstrs((I[j] <= MaxI for j in warehouse), "maximum inventory year 1") # ending inventory for warehouse j in year y

### alloy
for i in plant:
    model1.addConstr(sum(X[i,j] for j in warehouse)* 4.7 <= maxalloy, "alloy constraint for each year each plants")

model1.addConstrs((I[j]<=12000 for j in warehouse),"MAX") # max inventory for warehouses
for j in warehouse:
    model1.addConstr(sum(X[i,j] for i in plant) <=12000,"max flow in for each warehouse per year") 
for k in retail:
     model1.addConstr(sum(Y[j,k] for j in warehouse)<=12000,"max flow out for each warehouse per year")

model1.addConstrs((leftover[j] <= 4000 for j in warehouse), "remaining")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [11]:
#model1.addConstrs(C[i][j] >= O[i][j], "Construction") # only constructed before operating 
model1.addConstrs((R[i] == O[i] for i in plant), "operation year 1")

## plants i to warehouse j in year y shipping & capacity constraints

for i in plant:
    model1.addConstr(X.sum(i,'*')  <= Capacity[i-1]*O[i], "products from plant i to warehouse j within plant capacity")

for i in plant:
    model1.addConstr(O[i]==C[i])

## warehouse j to retail k in year y shipping constraints
for j in warehouse:
    model1.addConstr(sum(Y[j,k] for k in retail) <= sum(X[i,j] for i in plant), "warehouse shipping constraints")
model1.addConstrs((Y.sum('*',k) == demand[k-1] for k in retail), "Demand ") #demand constraint
model1.addConstrs((X.sum('*', j) <= Y.sum(j,'*') for j in warehouse),"flow conservation")

# inventory constraint
model1.addConstrs((I[j] <= MaxI for j in warehouse), "maximum inventory") # ending inventory for warehouse j in year y
### alloy
for i in plant:
    model1.addConstr(sum(X[i,j] for j in warehouse)* 4.7 <= maxalloy, "alloy constraint for each year each plants")

model1.addConstrs((I[j]<=12000 for j in warehouse),"MAX") # max inventory for warehouses
for j in warehouse:
    model1.addConstr(sum(X[i,j] for i in plant) <=12000,"max flow in for each warehouse per year") 
for k in retail:
     model1.addConstr(sum(Y[j,k] for j in warehouse)<=12000,"max flow out for each warehouse per year")

model1.addConstrs((leftover[j] <= 4000 for j in warehouse), "remaining")
for j in warehouse:
    model1.addConstr(X.sum('*', j) - leftover[j] == Y.sum(j,'*'),"flow conservation")



model1.optimize()
print(model1.ObjVal)
for var in model1.getVars():
    if(var.x>0):
        print(var.varName,var.x)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 438 rows, 105 columns and 2197 nonzeros
Model fingerprint: 0x2644c9d5
Variable types: 60 continuous, 45 integer (45 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [2e+01, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 6e+04]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 424 rows and 63 columns
Presolve time: 0.00s
Presolved: 14 rows, 42 columns, 82 nonzeros
Variable types: 40 continuous, 2 integer (2 binary)

Root relaxation: objective 2.235500e+06, 10 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2235500.00    0    1          - 2235500.00      -     -    0s

Cutting planes:
  MIR: 1
  Flow co

AttributeError: Unable to retrieve attribute 'ObjVal'

In [ ]:
Construction needs to occur either at or before the Operating variable in time.
For example:
Operating(Year1) = Construction(Year1)
Operating(Year2) = Construction(Year1) + Construction(Year2)
Operating(Year3) = Construction(Year1) + Construction(Year2) + Construction(Year3)
